In [1]:
!python --version

Python 3.7.12


In [1]:
%%capture

#enable the process bar
!conda install -y -c conda-forge ipywidgets
!jupyter nbextension enable --py widgetsnbextension

!conda install -y pytorch torchaudio cudatoolkit=10.2 -c pytorch
!pip install      datasets==1.13.3
!pip install      transformers==4.11.3
!pip install      librosa
!pip install      jiwer
!pip install      s3fs

In [2]:
%%capture 
!pip install boto3

In [3]:
import boto3
import datasets

In [12]:
# my_session = boto3.session.Session()
# s3 = my_session.client('s3')

In [5]:
# insert AWS keys here


In [6]:
s3 = boto3.resource(
service_name='s3',
region_name='eu-west-1',
aws_access_key_id= data['aws_access_key_id'],
aws_secret_access_key= data['aws_secret_access_key'])


In [2]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "tr", split="train+validation",cache_dir='/home/ec2-user/SageMaker/dataset')
common_voice_test = load_dataset("common_voice", "tr", split="test",cache_dir='/home/ec2-user/SageMaker/dataset/')

Reusing dataset common_voice (/home/ec2-user/SageMaker/dataset/common_voice/tr/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)
Reusing dataset common_voice (/home/ec2-user/SageMaker/dataset/common_voice/tr/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [3]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [4]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 3478
})

In [5]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [6]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)


  0%|          | 0/3478 [00:00<?, ?ex/s]

Loading cached processed dataset at /home/ec2-user/SageMaker/dataset/common_voice/tr/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9/cache-1782cb846a879a9f.arrow


In [14]:
common_voice_test

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 1647
})

In [7]:
def replace_hatted_characters(batch):
    batch["sentence"] = re.sub('[â]', 'a', batch["sentence"])
    batch["sentence"] = re.sub('[î]', 'i', batch["sentence"])
    batch["sentence"] = re.sub('[ô]', 'o', batch["sentence"])
    batch["sentence"] = re.sub('[û]', 'u', batch["sentence"])
    return batch

common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)


  0%|          | 0/3478 [00:00<?, ?ex/s]

Loading cached processed dataset at /home/ec2-user/SageMaker/dataset/common_voice/tr/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9/cache-e747932f924194e0.arrow


In [8]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 3478
})

In [9]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [10]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 3478
})

In [12]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
import json


In [21]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)
s3.Object('commonvoicesdataset', 'Turkish/vocab.json').put(Body=json.dumps(vocab_dict,indent = 3))

{'ResponseMetadata': {'RequestId': '6HNAAPQB4K1D381P',
  'HostId': 'KHEOQupY1PiMo27h8WPQTOVwW4SQVsWMxK+fNxuXn1U1u1V6ShQtmfvY20/04tMWGfcG38twd6c=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'KHEOQupY1PiMo27h8WPQTOVwW4SQVsWMxK+fNxuXn1U1u1V6ShQtmfvY20/04tMWGfcG38twd6c=',
   'x-amz-request-id': '6HNAAPQB4K1D381P',
   'date': 'Mon, 21 Feb 2022 11:25:57 GMT',
   'etag': '"fa5fc16d6b1c7d6f9f026e9a65eb4127"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"fa5fc16d6b1c7d6f9f026e9a65eb4127"'}

In [13]:
# this tokenizer is using the previous json file/ so becareful what u load
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [25]:
######

# save it to s3 in a folder for the language you want next to the json file 
#####



tokenizer.save_pretrained('/home/ec2-user/SageMaker')



('/home/ec2-user/SageMaker/tokenizer_config.json',
 '/home/ec2-user/SageMaker/special_tokens_map.json',
 '/home/ec2-user/SageMaker/vocab.json',
 '/home/ec2-user/SageMaker/added_tokens.json')

In [14]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [15]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [24]:
%pwd

'/home/ec2-user/SageMaker/STT'

In [16]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test  = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [17]:
common_voice_train

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 3478
})

In [18]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [19]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, num_proc=4)

In [20]:
common_voice_train


Dataset({
    features: ['input_values', 'labels'],
    num_rows: 3478
})

In [25]:
aws_access_key_id=data['aws_access_key_id']
aws_secret_access_key=data['aws_secret_access_key']
s3  = boto3.resource('s3',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)
s3c = boto3.client('s3',aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key)

In [ ]:

#from datasets.filesystems import S3FileSystem 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 32.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 KB 26.5 MB/s eta 0:00:00
  Created wheel for aiobotocore: filename=aiobotocore-2.1.1-py3-none-any.whl size=55483 sha256=bba75f387baccedfbd6e1f48c2e9a5a438ea3a2588aa7bbca3cfa2c78c46c080
  Stored in directory: /home/ec2-user/.cache/pip/wheels/52/fd/d3/e85dd95a98c12426ed6864dc24a408c3ca3253d4387b2f1cce
Successfully built aiobotocore
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.0
    Uninstalling botocore-1.24.0:
      Successfully uninstalled botocore-1.24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following d

In [26]:
import s3fs

In [28]:

key = data['aws_access_key_id']
secret = data['aws_secret_access_key']

In [29]:
s3fsystems = s3fs.S3FileSystem(key=key, secret=secret)  

In [30]:
common_voice_train.save_to_disk('s3://commonvoicesdataset/Turkish/train_set',fs=s3fsystems)

In [31]:
common_voice_test.save_to_disk('s3://commonvoicesdataset/Turkish/test_set',fs=s3fsystems)

In [32]:
common_voice_test

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1647
})

In [37]:
# from datasets import load_from_disk

In [38]:
# test = load_from_disk('s3://commonvoicesdataset/turkish/test',fs=s3fsystems)  

In [39]:
# train = load_from_disk('s3://commonvoicesdataset/turkish/train',fs=s3fsystems) 

In [ ]:
#dataset1 = load_from_disk('s3://commonvoicesdataset/test',fs=s31)  


In [40]:
# import torch

# from dataclasses import dataclass, field
# from typing import Any, Dict, List, Optional, Union

# @dataclass
# class DataCollatorCTCWithPadding:
#     """
#     Data collator that will dynamically pad the inputs received.
#     Args:
#         processor (:class:`~transformers.Wav2Vec2Processor`)
#             The processor used for proccessing the data.
#         padding (:obj:`bool`, :obj:`str` or :class:`a~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
#             Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
#             among:
#             * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
#               sequence if provided).
#             * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
#               maximum acceptable input length for the model if that argument is not provided.
#             * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
#               different lengths).
#         max_length (:obj:`int`, `optional`):
#             Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
#         max_length_labels (:obj:`int`, `optional`):
#             Maximum length of the ``labels`` returned list and optionally padding length (see above).
#         pad_to_multiple_of (:obj:`int`, `optional`):
#             If set will pad the sequence to a multiple of the provided value.
#             This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
#             7.5 (Volta).
#     """

#     processor: Wav2Vec2Processor
#     padding: Union[bool, str] = True
#     max_length: Optional[int] = None
#     max_length_labels: Optional[int] = None
#     pad_to_multiple_of: Optional[int] = None
#     pad_to_multiple_of_labels: Optional[int] = None

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         # split inputs and labels since they have to be of different lenghts and need
#         # different padding methods
#         input_features = [{"input_values": feature["input_values"]} for feature in features]
#         label_features = [{"input_ids": feature["labels"]} for feature in features]

#         batch = self.processor.pad(
#             input_features,
#             padding=self.padding,
#             max_length=self.max_length,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors="pt",
#         )
#         with self.processor.as_target_processor():
#             labels_batch = self.processor.pad(
#                 label_features,
#                 padding=self.padding,
#                 max_length=self.max_length_labels,
#                 pad_to_multiple_of=self.pad_to_multiple_of_labels,
#                 return_tensors="pt",
#             )

#         # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         batch["labels"] = labels

#         return batch

In [41]:
# import numpy  as np
# data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
# wer_metric = load_metric("wer")

# def compute_metrics(pred):
#     pred_logits = pred.predictions
#     pred_ids = np.argmax(pred_logits, axis=-1)

#     pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

#     pred_str = processor.batch_decode(pred_ids)
#     # we do not want to group tokens when computing the metrics
#     label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

#     wer = wer_metric.compute(predictions=pred_str, references=label_str)

#     return {"wer": wer}

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1647
})

In [42]:
# from transformers import Wav2Vec2ForCTC

# model = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-xls-r-300m", #facebook/wav2vec2-xls-r-300m
#     attention_dropout=0.1,
#     hidden_dropout=0.1,
#     feat_proj_dropout=0.0,
#     mask_time_prob=0.05,
#     layerdrop=0.1,
#     ctc_loss_reduction="mean", 
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer)
# )

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [44]:
# model.freeze_feature_extractor()

In [45]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(
#   # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
#   output_dir="./Turkish",
#   group_by_length=True,
#   per_device_train_batch_size=2,
#   gradient_accumulation_steps=2,
#   evaluation_strategy="steps", 
#   gradient_checkpointing=True,
#   num_train_epochs=1,
#   fp16=True,
#   save_steps=400,
#   eval_steps=400,
#   logging_steps=400,
#   learning_rate=3e-4,
#   warmup_steps=500,
#   save_total_limit=2,
# )

In [46]:
# from transformers import Trainer

# trainer = Trainer(
#     model=model,
#     data_collator=data_collator,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=train,
#     eval_dataset=test,
#     tokenizer=processor.feature_extractor,
# )

Using amp fp16 backend


In [90]:
# !nvidia-smi

Wed Feb 16 11:54:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   47C    P0    26W /  70W |   5455MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [85]:
# !sudo kill -9 3288                 

In [47]:
# trainer.train()

***** Running training *****
  Num examples = 3478
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 869
/home/ec2-user/anaconda3/envs/s2t/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811756765/work/torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)
/home/ec2-user/anaconda3/envs/s2t/lib/python3.7/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorre

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [1]:
# %pwd

'/home/ec2-user/SageMaker/STT'

In [2]:
# !aws s3 cp --recursive ./Turkish/ s3://commonvoicesdataset/Turkish//

upload failed: Turkish/checkpoint-400/preprocessor_config.json to s3://bucketname/checkpoint-400/preprocessor_config.json An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
upload failed: Turkish/checkpoint-400/scaler.pt to s3://bucketname/checkpoint-400/scaler.pt An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
upload failed: Turkish/checkpoint-400/trainer_state.json to s3://bucketname/checkpoint-400/trainer_state.json An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
upload failed: Turkish/checkpoint-400/rng_state.pth to s3://bucketname/checkpoint-400/rng_state.pth An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
upload failed: Turkish/checkpoint-400/scheduler.pt to s3://bucketname/checkpoint-400/scheduler.pt An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
upload failed: Turkish/checkpoint-400/config.json to 